In [1]:
import tensorflow as tf
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import re

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def retrieve_data(paths):
    sentence_spectrograph = {}
    for path in paths:
        speakers = os.listdir(path)
        print(path)
        for speaker in speakers:
            chapter_path = path + '\{}'.format(speaker)
            chapters = os.listdir(chapter_path)
            sentence_spectrograph[speaker] = []
            for chapter in chapters:
                main_path = chapter_path + '\{}'.format(chapter)
                main = os.listdir(main_path)
                data_pair = []
                for data in main[2:]:
                    data_path = main_path + '\{}'.format(data)
                    if(data.split('.')[1] == 'original' and data.split('.')[1] == 'txt'):
                        sentence = open(data_path, 'r').read()
                        data_pair.append(sentence)
                    elif(data.split('.')[1] == 'wav'):
                        sample_rate, samples = wavfile.read(data_path)
                        frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
                        data_pair.append(spectrogram)
                    
                    if(len(data_pair) == 2):
                        sentence_spectrograph[speaker].append(data_pair)
                        data_pair = []
            
    return sentence_spectrograph

In [4]:
training_paths = [r'C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Training\train-clean-360']

test_paths = [r'C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Test\test-clean', 
              r'C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Test\test-other']

validation_paths = [r'C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Development\dev-clean', 
              r'C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Development\dev-other']

In [5]:
def preprocess_sentences(data):
    new_data = {}
    for speaker in data.keys():
        data_pairs = data[speaker]
        new_data[speaker] = []
        for data_pair in data_pairs:
            sentence = data_pair[0]
            sentence = re.sub('([.,!?()"])', r' \1 ', sentence)
            sentence = sentence.replace("  ", " ")
            sentence = sentence.strip()
            new_data_pair = [sentence, data_pair[1]]
            new_data[speaker].append(new_data_pair)
    return new_data

In [6]:
training_data = retrieve_data(training_paths)
validation_data = retrieve_data(validation_paths)
test_paths = retrieve_data(test_paths)

C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Training\train-clean-360
C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Development\dev-clean
C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Development\dev-other
C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Test\test-clean
C:\Users\tanus\Deep Learning\Current Datasets\LibriTTS\Test\test-other
